In [1]:
import cv2
import mediapipe as mp
import os
from tqdm import tqdm
import pandas as pd
import time

In [132]:
DATA_PATH = '/media/navid/e068187d-d49f-4d11-8083-d6b72b15843d/data_process_coordinate_extract/100_Class_Data_Studio/SRITY/STUDIO/berano'
EXCEL_NAME = 'occlusion_test.xlsx'
VIDEO_SAVE_DIR = 'jana_output'

In [3]:
def get_all_video_paths(path):
    video_paths = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.mp4'):
                video_paths.append(os.path.join(root, file))
    return video_paths

In [4]:
def write_to_csv(data, path):
    df = pd.DataFrame(data)
    df.to_excel(path)

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [133]:
# file_paths = []
# frame_counts = []
# frame_numbers = []
# occlusion_comments = []
# left_hand_coordinates_count = []
# right_hand_coordinates_count = []

videos = get_all_video_paths(DATA_PATH)

In [134]:
if not os.path.exists(VIDEO_SAVE_DIR):
    os.makedirs(VIDEO_SAVE_DIR)

In [ ]:
# for video_path in tqdm(videos[:5]):
#     cap = cv2.VideoCapture(video_path)
#     frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     while cap.isOpened():
#         success, image = cap.read()
#         if not success:
#             break
#         image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         results = hands.process(image_rgb)
        
#         if results.multi_hand_landmarks:
#             for hand_landmarks in results.multi_hand_landmarks:
#                 # Check occlusion of fingers
#                 thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
#                 index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
#                 middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
#                 ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
#                 pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]       

#                 # Compare the z-coordinates of finger tips
#                 if thumb_tip.z < index_finger_tip.z and thumb_tip.z < middle_finger_tip.z and thumb_tip.z < ring_finger_tip.z and thumb_tip.z < pinky_tip.z:
#                     occlusion_comments.append('Thumb is occluded')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)

#                 if index_finger_tip.z < thumb_tip.z and index_finger_tip.z < middle_finger_tip.z and index_finger_tip.z < ring_finger_tip.z and index_finger_tip.z < pinky_tip.z:
#                     occlusion_comments.append('Index finger is occluded')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)

#                 if middle_finger_tip.z < thumb_tip.z and middle_finger_tip.z < index_finger_tip.z and middle_finger_tip.z < ring_finger_tip.z and middle_finger_tip.z < pinky_tip.z:
#                     occlusion_comments.append('Middle finger is occluded')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)

#                 if ring_finger_tip.z < thumb_tip.z and ring_finger_tip.z < index_finger_tip.z and ring_finger_tip.z < middle_finger_tip.z and ring_finger_tip.z < pinky_tip.z:
#                     occlusion_comments.append('Ring finger is occluded')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)

#                 if pinky_tip.z < thumb_tip.z and pinky_tip.z < index_finger_tip.z and pinky_tip.z < middle_finger_tip.z and pinky_tip.z < ring_finger_tip.z:
#                     occlusion_comments.append('Pinky is occluded')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)
#                 else:
#                     occlusion_comments.append('No occlusion')
#                     file_paths.append(video_path)
#                     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     frame_counts.append(frame_count)
#         else:
#             occlusion_comments.append('No hands detected')
#             file_paths.append(video_path)
#             frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
#             frame_counts.append(frame_count)
#         break

# # df = pd.DataFrame({
# #     'File Path': file_paths,
# #     'Frame Number': frame_numbers,
# #     'Occlusion Comments': occlusion_comments,
# # })
# # write_to_excel(df, EXCEL_NAME)

# print(len(file_paths))
# print(len(frame_numbers))
# print(len(occlusion_comments))
# print(len(frame_counts))


In [135]:
frame_numbers = []
hand_landmarks_count = []
file_paths = []

import random

# videos = random.sample(videos, 200)

for video_path in tqdm(videos):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    file_name = video_path.split('/')[-1]
    out_path = os.path.join(VIDEO_SAVE_DIR, file_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(out_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        landmarks_list = []
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(
                        color=(0, 255, 255), thickness=1, circle_radius=1
                    ),
                    mp_drawing.DrawingSpec(
                        color=(255, 255, 0), thickness=1, circle_radius=1
                    ),
                )
                landmarks_list.append(hand_landmarks.landmark)

        # if len(landmarks_list) == 2:
        #     hand_landmarks_count.append(len(landmarks_list[0]))
        #     frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
        #     file_paths.append(video_path)


        cv2.putText(image, "Hands Count: " + str(len(landmarks_list)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # time.sleep(.5)

        output.write(image)
        output.write(image)
        output.write(image)
        output.write(image)
        output.write(image)
        output.write(image)
        output.write(image)
        # # Display processed frame
        cv2.imshow("Frame", image)
        if cv2.waitKey(1) == ord("q"):
            break

cap.release()
output.release()
cv2.destroyAllWindows()
# data = {
#     'File Path': file_paths,
#     'Frame Number': frame_numbers,
#     'Hand Landmarks Count': hand_landmarks_count,
# }

# write_to_csv(data, EXCEL_NAME)

                

100%|██████████| 17/17 [05:44<00:00, 20.27s/it]
